In [1]:
import os
from src.pix2pix_turbo import Pix2Pix_Turbo
from PIL import Image
import torch
from torchvision import transforms
import torchvision.transforms.functional as F

In [2]:
models = {
    0: 'output/pix2pix_turbo/angel_0_go/checkpoints/model_1001.pkl', # 180.44
    1: 'output/pix2pix_turbo/angel_augmentation2_1/checkpoints/best_model.pkl', # 185.57
    2: 'output/pix2pix_turbo/angel_2/checkpoints/model_2001.pkl', # 218.61
    3: 'output/pix2pix_turbo/angel_augmentation_3/checkpoints/best_model.pkl', # 193.63
    4: 'output/pix2pix_turbo/angel_4/checkpoints/model_4501.pkl', # 187.45
    5: 'output/pix2pix_turbo/angel_5/checkpoints/model_3001.pkl' # 173.64
}

In [3]:
src_dir = 'data/oct/validation_set/val/CF/'
dst_dir = 'data/oct/generated_OCT_images_DIR'
dst_512_dir = 'data/oct/generated_OCT_images_DIR_512'

In [4]:
def generate_images(model_index, src_dir, dst_dir, dst_512_dir):
    model = Pix2Pix_Turbo(pretrained_path=models[model_index])
    model.set_eval()
    #model.half()

    for _, dirs, files in os.walk(src_dir):
        
        for f in files:
            f_path = os.path.join(src_dir, f)
            input_image = Image.open(f_path).convert('RGB')
            new_width = input_image.width - input_image.width % 8
            new_height = input_image.height - input_image.height % 8
            input_image = input_image.resize((512, 512), Image.LANCZOS)
            bname = os.path.basename(f)

            # translate the image
            with torch.no_grad():
                c_t = F.to_tensor(input_image).unsqueeze(0).cuda()
                #c_t = c_t.half()
                if 'L' in f:
                    test_prompts = "This is the color fundus image of left eye, please generate corresponding optical coherence tomography(OCT) image"
                else:
                    test_prompts = "This is the color fundus image of right eye, please generate corresponding optical coherence tomography(OCT) image"

                output_image = model(c_t, test_prompts)
                output_pil = transforms.ToPILImage()(output_image[0].cpu() * 0.5 + 0.5)
                
            dst_dir_path = os.path.join(dst_dir, bname.split('.')[0])
            dst_512_dir_path = os.path.join(dst_512_dir, bname.split('.')[0])
            if not os.path.exists(dst_dir_path):
                os.makedirs(dst_dir_path)
            if not os.path.exists(dst_512_dir_path):
                os.makedirs(dst_512_dir_path)
            output_pil.save(os.path.join(dst_512_dir_path, f"{bname.split('.')[0]}_{model_index}.jpg"))
            output_pil = output_pil.resize((448, 320), Image.LANCZOS)
            output_pil.save(os.path.join(dst_dir_path, f"{bname.split('.')[0]}_{model_index}.jpg"))
    model.release_memory()

In [5]:
generate_images(3, src_dir, dst_dir, dst_512_dir+'_3')

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Initializing model with OWN weights


In [10]:
def generate_images_sequence(model_index, src_dir, dst_dir, dst_512_dir):
    """
    Generate images sequence using Pix2Pix_Turbo model and save the outputs.

    Args:
        model_index (int): Index of the model in the `models` list.
        src_dir (str): Path to the source directory containing input images.
        dst_dir (str): Path to the destination directory for resized images.
        dst_512_dir (str): Path to the destination directory for 512x512 images.
    """
    print(f"Loading model: {models[model_index]}")
    
    # Initialize the model
    model = Pix2Pix_Turbo(pretrained_path=models[model_index])
    model.set_eval()

    # Walk through the source directory
    for root, _, files in os.walk(src_dir):
        for file_name in files:
            file_path = os.path.join(root, file_name)

            # Ensure the file is an image
            if not file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                print(f"Skipping non-image file: {file_name}")
                continue

            try:
                # Load and preprocess the input image
                input_image = Image.open(file_path).convert('RGB')
                base_name = os.path.basename(file_name).split('_')[0]
                
                with torch.no_grad():
                    # Convert image to tensor and move to GPU
                    c_t = F.to_tensor(input_image).unsqueeze(0).cuda()
                    test_prompt = "generate OCT images rotated 30 degrees clockwise"

                    # Generate output using the model
                    output_tensor = model(c_t, test_prompt)
                    output_pil = transforms.ToPILImage()(output_tensor[0].cpu() * 0.5 + 0.5)

                # Save the 512x512 output
                dst_512_path = os.path.join(dst_512_dir, base_name)
                os.makedirs(dst_512_path, exist_ok=True)
                output_pil.save(os.path.join(dst_512_path, f"{base_name}_{model_index}.jpg"))

                # Resize and save the 448x320 output
                resized_output = output_pil.resize((448, 320), Image.LANCZOS)
                dst_resized_path = os.path.join(dst_dir, base_name)
                os.makedirs(dst_resized_path, exist_ok=True)
                resized_output.save(os.path.join(dst_resized_path, f"{base_name}_{model_index}.jpg"))

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Release model memory
    model.release_memory()
    print("Model memory released.")


In [11]:
for i in range(1, 6):
    generate_images_sequence(i, dst_512_dir + f'_{i-1}', dst_dir, dst_512_dir + f'_{i}')

Loading model: output/pix2pix_turbo/sequence_angel_1/checkpoints/best_model.pkl
Initializing model with OWN weights
Model memory released.
Loading model: output/pix2pix_turbo/sequence_angel_2/checkpoints/best_model.pkl
Initializing model with OWN weights
Model memory released.
Loading model: output/pix2pix_turbo/sequence_angel_3/checkpoints/best_model.pkl
Initializing model with OWN weights
Model memory released.
Loading model: output/pix2pix_turbo/sequence_angel_4/checkpoints/best_model.pkl
Initializing model with OWN weights
Model memory released.
Loading model: output/pix2pix_turbo/sequence_angel_5/checkpoints/best_model.pkl
Initializing model with OWN weights
Model memory released.


In [6]:
import os
import zipfile

def zip_directory(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                # 获取文件的完整路径
                file_path = os.path.join(root, file)
                # 在压缩包中存储的路径，去除父目录路径部分
                arcname = os.path.relpath(file_path, start=folder_path)
                # 添加文件到压缩包中
                zipf.write(file_path, arcname)
    print(f"Folder '{folder_path}' has been compressed into '{output_zip}'")

# 使用示例
zip_directory("data/oct/generated_OCT_images_DIR", "submit_data.zip")


Folder 'data/oct/generated_OCT_images_DIR' has been compressed into 'submit_data.zip'
